In [ ]:
import polars as pl
import sys
import os

sys.path.append(os.path.dirname(os.path.abspath('')))

from libraries.client_stashapp import get_stashapp_client, StashAppClient

stash = get_stashapp_client()
stash_client = StashAppClient()

In [2]:
all_performers = stash_client.get_performers()
eva_de_vil = all_performers.filter(pl.col("stashapp_name").str.contains("Eva de Vil")).to_dicts()[0]

In [ ]:
eva_de_vil["stashapp_stash_ids"]


In [4]:
sample_scenes = stash_client.find_scenes_by_performers([eva_de_vil["stashapp_id"]])

In [ ]:
sample_scenes.to_dicts()[0]["stashapp_files"][0]["path"]

In [ ]:
sample_scene = sample_scenes.to_dicts()[0]
sample_scene

In [ ]:
from libraries.FaceDatasetBuilder import FaceDatasetBuilder

# Initialize builder
builder = FaceDatasetBuilder()

def get_stashdb_performer(performer):
    """Extract StashDB ID and name from performer data"""
    for stash_id in performer["stashapp_performers_stash_ids"]:
        if stash_id["endpoint"] == "https://stashdb.org/graphql":
            return {
                'stash_id': stash_id["stash_id"],
                'name': performer["stashapp_performers_name"]
            }
    return None

# Process a scene
def process_scene(scene):
    # Get performers with their StashDB IDs
    performers = []
    for performer in scene["stashapp_performers"]:
        stashdb_info = get_stashdb_performer(performer)
        if stashdb_info:
            performers.append(f"{stashdb_info['stash_id']} - {stashdb_info['name']}")
    
    # Process the scene
    if performers:
        return builder.process_scene(
            video_path=scene["stashapp_primary_file_path"],
            scene_id=scene["stashapp_stashdb_id"],
            performers=performers
        )
    return 0

# Process your sample scene
faces_extracted = process_scene(sample_scene)
print(f"Extracted {faces_extracted} faces")

# Verify directories exist
import os
base_dir = "H:\\Faces\\dataset"
print("\nDirectory structure:")
for root, dirs, files in os.walk(base_dir):
    level = root.replace(base_dir, '').count(os.sep)
    indent = ' ' * 4 * level
    print(f"{indent}{os.path.basename(root)}/")
    if level < 2:  # Only show first two levels
        for d in dirs:
            print(f"{indent}    {d}/")

In [ ]:
# Later, verify faces
builder.verify_face('scene123_frame_0001_face_0', 'performer1')  # Verify
builder.verify_face('scene123_frame_0001_face_1', is_rejected=True)  # Reject